## Reglas
* Jorge Leonardo Raba Gonzalez
* Oscar A. Mendez Aguirre

In [166]:
import os
import re
import csv
import pprint
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("data.csv",  encoding='utf8', low_memory=False )

In [4]:
data.shape

(4410, 12)

In [5]:
data.tail(2)

,_id,collective,date,file_id,judicature,participants,report_receipt_at,sentence_id,text,topic,url,vocabulary
4408,corteconstitucional_A. 175A/19,"{""entity"": {""$oid"": ""5fa5017c5821d011a2a92372""}}",2019-04-03T00:00:00.000Z,T-7071794,"[{""name"": ""ALBERTO ROJAS RÍOS"", ""rol"": ""PONENT...","[{""name"": ""DARIO GOMEZ SUAREZ"", ""role"": ""PLAIN...",2019-07-08,A. 175A/19,Auto 175A/19\n \n \nReferencia: Expediente T-7...,El accionante le solicitó a la Corporación dec...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 7}, {""text"": ""referen..."
4409,corteconstitucional_A. 121/19,"{""entity"": {""$oid"": ""5fa501875821d011a2a93763""}}",2019-03-13T00:00:00.000Z,T-6843600,"[{""name"": ""GLORIA STELLA ORTIZ DELGADO"", ""rol""...","[{""name"": ""COMISION INTERECLESIAL DE JUSTICIA ...",2019-03-21,A. 121/19,Auto\r\n121/19\n \n \nReferencia: expediente T...,Con el presente auto se suspenden los términos...,https://www.corteconstitucional.gov.co/relator...,"[{""text"": ""auto"", ""qty"": 17}, {""text"": ""refere..."


In [6]:
data = data.drop(["judicature", "participants", "collective", "_id", "topic", "url", "vocabulary", "date"], axis=1)

In [7]:
data.head()

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...
1,T-3183107,2012-05-18,T-016/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-016/...
2,T-3187308,2013-05-07,T-014/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia\r\nT-014/...
3,T-3182540,2012-05-18,T-015/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-015/12\...
4,T-2719663,2011-05-10,T-012/11,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-012/11\...


In [8]:
data.head(1)

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,REPÚBLICA DE COLOMBIA\n\n\nSentencia T-017/12\...


In [9]:
data["text"] = data["text"].str.lower()

In [10]:
len(data["text"])

4410

In [11]:
data

,file_id,report_receipt_at,sentence_id,text
0,T-3140852,2012-08-08,T-017/12,república de colombia\n\n\nsentencia t-017/12\...
1,T-3183107,2012-05-18,T-016/12,república de colombia\n\n\nsentencia\r\nt-016/...
2,T-3187308,2013-05-07,T-014/12,república de colombia\n\n\nsentencia\r\nt-014/...
3,T-3182540,2012-05-18,T-015/12,república de colombia\n\n\nsentencia t-015/12\...
4,T-2719663,2011-05-10,T-012/11,república de colombia\n\n\nsentencia t-012/11\...
...,...,...,...,...
4405,T-6403234,2019-04-08,A. 177/19,auto 177/19\n \nreferencia:\r\nexpediente t-64...
4406,T-6945381,2019-05-27,A. 177A/19,auto\r\n177a/19\n \n \nreferencia:\r\nexpedien...
4407,T-7064392 Y T-7073283,2019-04-08,SU.139/19,sentencia\r\nsu139/19 \n \naccion de tutela co...
4408,T-7071794,2019-07-08,A. 175A/19,auto 175a/19\n \n \nreferencia: expediente t-7...


In [12]:
new_data = pd.DataFrame([], columns = ['file_id', 'antecedentes' , 'decision'])

In [13]:
new_data

,file_id,antecedentes,decision


In [14]:
def quitar_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [15]:
def get_segment(data):
    lista_df = []
    for index, row in data.iterrows():
        texto = re.split(
            r'[\n]+[(^i|^v|)]+\.\s{0,}[(\w)]{0,}?', 
            quitar_tildes(row["text"])
        )
        if len(texto) > 4:
            lista_df.append({"file_id": row["file_id"] , "antecedentes": texto[1], "decision": texto[4]})

    return lista_df

In [16]:
new_data = pd.DataFrame(get_segment(data), columns = ["file_id", 'antecedentes' , 'decision'])

In [17]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes\n \n1. hechos\n \n1. el 27 de jul...,decision\n \ncon\r\nfundamento en las consider...
1,T-2719663,antecedentes\n \n1. hechos\n \nel defensor ...,"decision\n \nen merito de lo expuesto, la\r\ns..."
2,T-3176539,antecedentes\n \nel ocho (8) de junio de dos m...,"decision\n \nen merito de lo expuesto, la sala..."
3,T-3180007,"antecedentes\n \nel senor juan antonio, a trav...",consideraciones de la corte constitucional\n \...
4,T-2764991 Y 2822307,antecedentes\n \n1. hechos\n \nlos hechos rela...,"decision \n \nen merito de lo expuesto, la sal..."
...,...,...,...
1857,T-7128811,antecedentes\n \n1. hechos\n \n\n1.1 la senora...,que no esten\r\nexpresamente incluidos en el p...
1858,T-6960341,antecedentes \n \n1. hechos y relato contenido...,pruebas decretadas en sede de revision \n \nel...
1859,T-7147012,antecedentes\n \nherley\r\nramirez alzate pres...,existencia\r\nde un peligro especifico e indiv...
1860,T-7103649,antecedentes\n \na. hechos y pretensiones\n \n...,"decision\n \nen merito de lo expuesto, la sala..."


In [18]:
new_data.shape

(1862, 3)

In [19]:
new_data.to_csv('splid_data.csv', sep=",",quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')

In [20]:
new_data.to_csv(r'splid_data.txt', header=None, index=None, sep=' ', mode='a')

In [21]:
def quitar_espacios(text):
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    text = re.sub('\xa0', ' ', text)

    return text

new_data["decision"] = new_data.decision.apply(lambda x: quitar_espacios(x))
new_data["antecedentes"] = new_data.antecedentes.apply(lambda x: quitar_espacios(x))

In [22]:
def quita_signos(text):
    puntuacion = string.punctuation
    for item in puntuacion:
        text = text.replace(item, "")
    return text

new_data["decision"] = pd.DataFrame(new_data.decision.apply(lambda x: quita_signos(x)))
new_data["antecedentes"] = pd.DataFrame(new_data.antecedentes.apply(lambda x: quita_signos(x)))

In [23]:
new_data[1:2]["antecedentes"]

1    antecedentes   1    hechos   el defensor del p...
Name: antecedentes, dtype: object

In [24]:
new_data["antecedentes"]

0       antecedentes   1 hechos   1 el 27 de julio de ...
1       antecedentes   1    hechos   el defensor del p...
2       antecedentes   el ocho 8 de junio de dos mil  ...
3       antecedentes   el senor juan antonio a traves ...
4       antecedentes   1 hechos   los hechos relatados...
                              ...                        
1857    antecedentes   1 hechos    11 la senora karina...
1858    antecedentes    1 hechos y relato contenidos e...
1859    antecedentes   herley  ramirez alzate presento...
1860    antecedentes   a hechos y pretensiones   1  la...
1861    antecedentes   1     hechos relevantes   11   ...
Name: antecedentes, Length: 1862, dtype: object

In [25]:
new_data["decision"][50][0:2000]

'decision   en merito de lo expuesto la  sala novena de revision de la corte constitucional administrando justicia en  nombre del pueblo y por mandato de la constitucion     resuelve   primero revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado en su lugar conceder  la tutela por violacion del derecho al debido proceso   segundo dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civilfamilia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  sa contra la empresa construcciones dihago ltda representada por alejandro  habeych gonzalez y contra las personas ana maria barrios quevedo susana  magd

In [513]:
new_data["decision"][50][1:1000]

'ecision   en merito de lo expuesto, la  sala novena de revision de la corte constitucional, administrando justicia en  nombre del pueblo y por mandato de la constitucion,     resuelve:   primero.- revocar la sentencia  proferida el 27 de septiembre  de 2011 por la sala de casacion laboral de la  corte suprema de justicia que a su turno habia confirmado la sentencia dictada  el 10 de agosto de 2011 por sala de casacion civil de la corte suprema de  justicia que habia negado el amparo constitucional solicitado. en su lugar, conceder  la tutela por violacion del derecho al debido proceso.   segundo.- dejar sin efecto  las actuaciones surtidas por el juzgado cuarto civil del circuito de santa  marta y por la sala civil-familia del tribunal superior de la misma ciudad en  el proceso ejecutivo hipotecario seguido por el ahorramas hoy banco av villas  s.a. contra la empresa construcciones dihago ltda. representada por alejandro  habeych gonzalez y contra las personas ana maria barrios queved

In [36]:
def get_list_sentenses_by_model(text_count):
    list_sentenses = []
    for sentense in sd_model.fullAnnotate(text_count)[0]["sentences"]:
        list_sentenses.append(sentense.result)
    return list_sentenses

In [37]:
def extraer_articulos(sentences, topic):
    lista_df = []
    for item in sentences:
        regx = f"({topic}|{topic}s)[ ]([\s\S]*?)\."
        expretion = re.compile(r"{}".format(regx))
        result = expretion.findall(item)
        for item_normalize in list(map(" ".join, result)):
            lista_df.append(item_normalize)
    return lista_df

In [38]:

def get_list_sentenses():
    list_sentenses = []
    for sentense in sd_model.fullAnnotate(text)[0]["sentences"]:
        list_sentenses.append(sentense.result)
    return list_sentenses

In [39]:
new_dataframe_to_graph = pd.DataFrame(columns = ["folios", "reference"])

In [40]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes 1. hechos 1. el 27 de julio d...,decision con fundamento en las consideracio...
1,T-2719663,antecedentes 1. hechos el defensor del ...,"decision en merito de lo expuesto, la sala ..."
2,T-3176539,antecedentes el ocho (8) de junio de dos mil...,"decision en merito de lo expuesto, la sala t..."
3,T-3180007,"antecedentes el senor juan antonio, a traves...",consideraciones de la corte constitucional 1...
4,T-2764991 Y 2822307,antecedentes 1. hechos los hechos relatado...,"decision en merito de lo expuesto, la sala ..."
...,...,...,...
1857,T-7128811,antecedentes 1. hechos 1.1 la senora kari...,que no esten expresamente incluidos en el pbs...
1858,T-6960341,antecedentes 1. hechos y relato contenidos ...,pruebas decretadas en sede de revision el ...
1859,T-7147012,antecedentes herley ramirez alzate presento...,existencia de un peligro especifico e individ...
1860,T-7103649,antecedentes a. hechos y pretensiones 1. ...,"decision en merito de lo expuesto, la sala s..."


# Seccion DECISION [sentencias, articulos, folios, decretos]

In [41]:
text = new_data["decision"][28]

In [42]:
decision_new_data_folio = pd.DataFrame(columns = ["folios", "reference"])
decision_new_data_articulo = pd.DataFrame(columns = ['articulos', "reference"])
decision_new_data_decreto = pd.DataFrame(columns = ['decretos', "reference"])
decision_new_data_sentencia = pd.DataFrame(columns = ['sentencias', "reference"])

In [43]:
decision_new_data_folio['folios'] = extraer_articulos(get_list_sentenses()[27:128], "folio")

In [44]:
decision_new_data_decreto['decretos'] = extraer_articulos(get_list_sentenses(), "decreto")

In [45]:
decision_new_data_articulo['articulos'] = extraer_articulos(get_list_sentenses(), "articulo")

In [46]:
decision_new_data_sentencia['sentencias'] = extraer_articulos(get_list_sentenses()[27:128], "sentencia")

In [47]:
decision_new_data_articulo.tail(30)

,articulos,reference
0,"articulos 86 y 241, numeral 9°, de la constit...",NaN
1,"articulo 29 superior, dispone que el debido p...",NaN
2,"articulo 209 superior), toda vez que a traves ...",NaN
3,"articulo 86 superior, la accion de tutela es u...",NaN
4,"articulo 6° del decreto 2591 de 1991, el cual...",NaN
5,articulo 49 del codigo contencioso administrat...,NaN
6,articulo 58 y el decreto distrital 531 de 23 ...,NaN
7,articulo 58 y el decreto distrital 531 de 23 d...,NaN
8,articulo 28 del c,NaN
9,"articulo 17 de la ley 1333 de 2009, la privo d...",NaN


In [48]:
folios = extraer_articulos(get_list_sentenses_by_model(new_data["decision"][28]), "folio")
folios

[]

In [49]:
decision_new_data_folio = pd.DataFrame(columns = ["folios"])
decision_new_data_articulo = pd.DataFrame(columns = ['articulos'])
decision_new_data_decreto = pd.DataFrame(columns = ['decretos'])
decision_new_data_sentencia = pd.DataFrame(columns = ['sentencias'])

def get_new_data(sentence):
    folios = extraer_articulos(get_list_sentenses_by_model(sentence), "folio")
    articulos = extraer_articulos(get_list_sentenses_by_model(sentence), "articulo")
    sentencias = extraer_articulos(get_list_sentenses_by_model(sentence), "sentencia")
    decretos = extraer_articulos(get_list_sentenses_by_model(sentence), "decreto")
    return folios, articulos, sentencias, decretos
    

In [50]:
dec = pd.DataFrame(columns = ["folios", "articulos", "sentencias", "decretos", "reference"])
for item in range(1, 1800):
    reference = new_data["file_id"][item]
    folios, articulos, sentencias, decretos = get_new_data(new_data["decision"][item])
    dec = dec.append(
        {
         "folios": articulos, 
         "articulos": articulos, 
         "sentencias": sentencias, 
         "decretos": decretos, 
         "reference": reference
        }, 
        ignore_index=True
    )
dec

,folios,articulos,sentencias,decretos,reference
0,"[articulo 36 del decreto 2591 de 1991, articul...","[articulo 36 del decreto 2591 de 1991, articul...",[sentencia y del expediente a la superintenden...,[decreto 2591 de 1991],T-2719663
1,"[articulo 36 del decreto 2591 de 1991, para l...","[articulo 36 del decreto 2591 de 1991, para l...",[sentencia proferida por el juzgado primero pe...,"[decreto 2591 de 1991, para los efectos alli ...",T-3176539
2,[articulos 86 y 241-9 de la constitucion poli...,[articulos 86 y 241-9 de la constitucion poli...,"[sentencia c-134 de 1994, en la que se declar...","[decreto 2591 de 1991, decreto 2591 de 1991, ...",T-3180007
3,[articulo 239 del codigo sustantivo del trabaj...,[articulo 239 del codigo sustantivo del trabaj...,[sentencia proferida el siete (7) de mayo de 2...,"[decreto 2591 de 1991, decreto 3135 de 1968, e...",T-2764991 Y 2822307
4,[articulos 86 y 241 numeral 9° de la constituc...,[articulos 86 y 241 numeral 9° de la constituc...,[sentencia proferida dentro del proceso de la ...,[decreto 2591 de 1991],T-3201980
...,...,...,...,...,...
1794,"[articulo 36 del decreto ley 2591 de 1991, art...","[articulo 36 del decreto ley 2591 de 1991, art...",[sentencias proferidas el tres (3) de septiemb...,"[decreto ley 2591 de 1991, decreto 2591 de 199...",T-7136911
1795,[articulo 55 (sic) del reglamento interno de ...,[articulo 55 (sic) del reglamento interno de ...,[sentencia t-284 de 2019[153] (expediente t-7...,[],T-7055614 Y OTRO ACUMULADOS
1796,[articulo 488 del codigo sustantivo del traba...,[articulo 488 del codigo sustantivo del traba...,"[sentencias t-426 de 2001 m, sentencias t-051 ...","[decreto 2591 de 1991, decreto 1730 de 2001: i...",T-7222037 Y OTRO ACUMULADOS
1797,"[articulos 86 y 241, numeral 9º, de la consti...","[articulos 86 y 241, numeral 9º, de la consti...",[sentencias proferidas dentro del proceso de l...,"[decreto 2591 de 1991, decreto 2591 de 1991 s...",T-6899349


In [51]:
dec["folios"][0][1]

'articulo 46 el estado, la sociedad y la familia  concurriran para la proteccion y la asistencia de las personas de la tercera  edad y promoveran su integracion a la vida activa y comunitaria'

In [52]:
decision_new_data_articulo.head()

,articulos


In [53]:
dec.shape

(1799, 5)

In [85]:
dec[1050:1060]

NameError: name 'dec' is not defined

In [83]:
dec.to_parquet('processed_data.gzip', compression='gzip')

NameError: name 'dec' is not defined

In [56]:
decision_new_data_decreto.head()

,decretos


In [57]:
decision_new_data_folio.head()

,folios


In [95]:
parquet_data.to_csv('processed_data.csv')

# Reglas de asociacion

In [26]:
import plotly.express as px
import nltk
import seaborn as sns
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

/home/oscfrayle/PycharmProjects/UNAL/entrega2md/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/oscfrayle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from efficient_apriori import apriori
import pyfpgrowth

In [30]:
new_data

,file_id,antecedentes,decision
0,T-3187308,antecedentes 1 hechos 1 el 27 de julio de ...,decision con fundamento en las consideracio...
1,T-2719663,antecedentes 1 hechos el defensor del p...,decision en merito de lo expuesto la sala p...
2,T-3176539,antecedentes el ocho 8 de junio de dos mil ...,decision en merito de lo expuesto la sala te...
3,T-3180007,antecedentes el senor juan antonio a traves ...,consideraciones de la corte constitucional 1...
4,T-2764991 Y 2822307,antecedentes 1 hechos los hechos relatados...,decision en merito de lo expuesto la sala n...
...,...,...,...
1857,T-7128811,antecedentes 1 hechos 11 la senora karina...,que no esten expresamente incluidos en el pbs...
1858,T-6960341,antecedentes 1 hechos y relato contenidos e...,pruebas decretadas en sede de revision el ...
1859,T-7147012,antecedentes herley ramirez alzate presento...,existencia de un peligro especifico e individ...
1860,T-7103649,antecedentes a hechos y pretensiones 1 la...,decision en merito de lo expuesto la sala se...


In [31]:
parquet_data = pd.read_parquet('processed_data.gzip')

In [32]:
parquet_data

,folios,articulos,sentencias,decretos,reference
0,"[articulo 36 del decreto 2591 de 1991, articul...","[articulo 36 del decreto 2591 de 1991, articul...",[sentencia y del expediente a la superintenden...,[decreto 2591 de 1991],T-2719663
1,"[articulo 36 del decreto 2591 de 1991, para l...","[articulo 36 del decreto 2591 de 1991, para l...",[sentencia proferida por el juzgado primero pe...,"[decreto 2591 de 1991, para los efectos alli ...",T-3176539
2,[articulos 86 y 241-9 de la constitucion poli...,[articulos 86 y 241-9 de la constitucion poli...,"[sentencia c-134 de 1994, en la que se declar...","[decreto 2591 de 1991, decreto 2591 de 1991, ...",T-3180007
3,[articulo 239 del codigo sustantivo del trabaj...,[articulo 239 del codigo sustantivo del trabaj...,[sentencia proferida el siete (7) de mayo de 2...,"[decreto 2591 de 1991, decreto 3135 de 1968, e...",T-2764991 Y 2822307
4,[articulos 86 y 241 numeral 9° de la constituc...,[articulos 86 y 241 numeral 9° de la constituc...,[sentencia proferida dentro del proceso de la ...,[decreto 2591 de 1991],T-3201980
...,...,...,...,...,...
1794,"[articulo 36 del decreto ley 2591 de 1991, art...","[articulo 36 del decreto ley 2591 de 1991, art...",[sentencias proferidas el tres (3) de septiemb...,"[decreto ley 2591 de 1991, decreto 2591 de 199...",T-7136911
1795,[articulo 55 (sic) del reglamento interno de ...,[articulo 55 (sic) del reglamento interno de ...,[sentencia t-284 de 2019[153] (expediente t-7...,[],T-7055614 Y OTRO ACUMULADOS
1796,[articulo 488 del codigo sustantivo del traba...,[articulo 488 del codigo sustantivo del traba...,"[sentencias t-426 de 2001 m, sentencias t-051 ...","[decreto 2591 de 1991, decreto 1730 de 2001: i...",T-7222037 Y OTRO ACUMULADOS
1797,"[articulos 86 y 241, numeral 9º, de la consti...","[articulos 86 y 241, numeral 9º, de la consti...",[sentencias proferidas dentro del proceso de l...,"[decreto 2591 de 1991, decreto 2591 de 1991 s...",T-6899349


In [33]:
def extraer_ley(text):
    ley = ""
    regx = f"(ley)( +)+([0-9]+)+( +)(de)+( +)([0-9]+)"
    expretion = re.compile(r"{}".format(regx))
    result = expretion.findall(text)
    if result:
        return ley.join(list(result[0]))
    return ""

In [128]:
def extraer_decreto(text):
    decreto = ""
    regx = f"((decreto)( )([0-9]+)( )+(de)+( )([0-9]+))"
    expretion = re.compile(r"{}".format(regx))
    result = expretion.search(text)
    if result:
        return decreto.join(list(result[0]))
    return ""

In [129]:
def extraer_decreto2(text):
    decretos_list = []
    decreto = ""
    decretos = ", "
    result = re.findall("((decreto)( )([0-9]+)( )+(de)+( )([0-9]+))",text)
    for item in result:
        temporal_list = list(item)
        
        if len(decreto.join(list(item))) > 15:
            decretos_list.append(decreto.join(list(item)).strip().replace(",", ""))
    return decretos.join(decretos_list)

In [117]:
def extraer_decreto(text):
    decretos_list = []
    decreto = ""
    decretos = ", "
    result = re.findall("((decreto)( )([0-9]+)( )+(de)+( )([0-9]+))",text)
    for item in result:
        temporal_list = list(item)
        
        if len(decreto.join(list(item))) > 15:
            decretos_list.append(decreto.join(list(item)).strip().replace(",", ""))
    return decretos.join(decretos_list)

In [36]:
ley_test = extraer_ley("decreto 992 de 1930, normas que no estaban vigentes  ya que habian sido subrogadas por el decreto ley 1355 de 1970")
print(ley_test)

ley 1355 de 1970


In [37]:
derecho_test = extraer_derecho("decreto 992 de 1930, normas que no estaban en los derecho a la salud vigentes ya que habian sido subrogadas por el decreto ley 1355 de 1970")
print(derecho_test)

derecho a la salud


In [38]:
decreto_test = extraer_decreto("decreto 992 de 1931, decreto 992 de 1930, decreto 99232 de 192330 normas que no estaban en los derecho fundamental a la salud vigentes ya que habian sido subrogadas por decreto 99233 de 192331, el decreto ley 1355 de 1970")
print(decreto_test)

decreto 992 de 1931, decreto 992 de 1930, decreto 99232 de 192330, decreto 99233 de 192331


In [39]:
def compose_partial_line(dataframe_column):
    diccionario_line = {}
    for item, key in parquet_data.iterrows():
        lista_full_text = []
        final_line = ""
        for line in key[dataframe_column]:
            list_line = []
            full_text = ","
            decreto_extract = extraer_decreto(line)
            if decreto_extract:
                list_line.append(extraer_decreto(line))
        if list_line:
            diccionario_line.update({key["reference"]: full_text.join(list_line).strip()})
    return diccionario_line
#compose_partial_line("decretos")

In [40]:
parquet_data.shape

(1799, 5)

In [41]:
ley_df = {}
lista_referencias = {}
for item, key in parquet_data.iterrows():
    lista_leyes_sentencias = []
    for line in key["sentencias"]:
        _ = extraer_ley(line)
        if _:
            lista_leyes_sentencias.append(
                _
            )
            lista_referencias.update({key["reference"]: lista_leyes_sentencias})
            
print(lista_referencias.keys())

dict_keys(['T-3229258', 'T-3170053', 'T-3163147', 'T-3242509', 'T-3255611', 'T-3257265', 'T-3368359', 'T-3336431', 'T-3331151', 'T-2934614', 'T-3038260', 'T-3383054', 'T-3364320', 'T-3339178', 'T-2706361', 'T-3093713', 'T-3439513', 'T-3101081', 'T-3077270', 'T-3066688', 'T-3078861', 'T-3407562', 'T-3052938', 'T-3011473', 'T-2545429 Y OTROS ACUMULADOS', 'T-2448215', 'T-3015372 Y OTROS ACUMULADOS', 'T-3408860', 'T-3433683', 'T-3001628', 'T-2595774', 'T-3037072 acumulado', 'T-2944866', 'T-3415529', 'T-3405496', 'T-2980419', 'T-3415130', 'T-2097348', 'T-2927070', 'T-3343947 Y OTRO ACUMULADOS', 'T-2910430', 'T-2864851', 'T-2674656', 'T-2753981', 'T-2983421', 'T-859-11', 'T-3302323', 'T-2176281', 'T-2719755', 'T-2992723', 'T-3339097', 'T-2899768', 'T-3008975 Y OTRO ACUMULADO', 'T-2815439', 'T-2892198', 'T-2906284', 'T-2635345', 'T-2902876', 'T-3247689', 'T-3336110', 'T-2482392', 'T-3257281', 'T-3206395', 'T-3199440', 'T-2839194 Y OTRO ACUMULADOS', 'T-2813791 Y OTROS ACUMULADOS', 'T-2857660',

In [42]:
def compose_full_line(dataframe_column):
    ley_df = {}
    diccionario_line = {}
    for item, key in parquet_data.iterrows():
        lista_full_text = []
        final_line = ""
        for line in key[dataframe_column]:
            list_line = []
            full_text = ","
            ley_extract = extraer_ley(line)
            if ley_extract:
                list_line.append(extraer_ley(line))
            derecho_extract = extraer_derecho(line)
            if derecho_extract:
                list_line.append(derecho_extract.strip())
            decreto_extract = extraer_decreto(line)
            if decreto_extract:
                list_line.append(extraer_decreto(line))
            if list_line and ley_extract and derecho_extract and decreto_extract:
            #if list_line:
                final_line = full_text.join(list_line)
                lista_full_text.append(final_line)
            list_line = []
        if lista_full_text:    
            print(lista_full_text,  key["reference"])

            
compose_full_line("sentencias") 

['ley 797 de 2003,derecho a la seguridad,decreto 3085 de 2007, decreto 3085 de  2007'] T-4808412
['ley 1448 de 2011,derechos fundamentales a la vida,decreto 1290 de 2008'] T-2687739
['ley 100 de 1993,derecho a la  pension,decreto 1213 de 1990'] T-3746878


# antecedentes [derechos, leyes, decretos]

In [130]:
count = 0
antecedentes_df = pd.DataFrame(columns = ["reference", "text"])
antecedentes_full_text = {}
for item, key in new_data.iterrows():
    item_derecho = extraer_derecho(key["antecedentes"])
    item_decreto = extraer_decreto(key["antecedentes"])
    item_ley = extraer_ley(key["antecedentes"])
    count = count + 1
    if item_derecho and item_decreto and item_ley:
        antecedentes_df = antecedentes_df.append({
            "reference": key["file_id"],
            "text": list(set([item_derecho.strip(), item_decreto.strip().replace(', , ,',''), item_ley.strip().replace(',,','')]))
        }, 
        ignore_index=True
        )
antecedentes_df

,reference,text
0,T-3187308,"[ley 100 de 1993, decreto 2196 de 2009, derech..."
1,T-2809578,"[decreto 094 de 1989, derechos fundamentales ..."
2,T-3170053,"[ley 57 de 1905, decreto 992 de 1993, derechos..."
3,T-3154181,"[ley 267 de 2000, derechos fundamentales al de..."
4,T-3215125,"[derecho a la pension, ley 100 de 1993, decret..."
...,...,...
394,T-7057599,"[ley 1151 de 2007, derechos fundamentales al ..."
395,T-7136220,"[ley 100 de 1993, decreto 2591 de 1991, derech..."
396,T-7139120,"[ley 860 de 2003, derechos fundamentales a la..."
397,T-7163048,"[decreto 624 de 1989, derechos fundamentales a..."


# decision [derechos, leyes, decretos]

In [131]:
count = 0
decision_df = pd.DataFrame(columns = ["reference", "text"])
decision_full_text = {}
for item, key in new_data.iterrows():
    item_derecho = extraer_derecho(key["decision"])
    item_decreto = extraer_decreto(key["decision"])
    item_ley = extraer_ley(key["decision"])
    count = count + 1
    if item_derecho and item_decreto and item_ley:
        decision_df = decision_df.append({
            "reference": key["file_id"],
            "text": list(set([item_derecho.strip(), item_decreto.strip().replace(', , ,',''), item_ley.strip().replace(',,','')])) #f"{item_derecho.strip()}, {item_decreto.strip()}, {item_ley.strip().replace(',','')}"
        }, 
        ignore_index=True
        )
decision_df

,reference,text
0,T-2719663,"[ley 1122 de 2007, derecho a la salud, decreto..."
1,T-3180007,"[ley 1098 de 2006, derecho a la igualdad, decr..."
2,T-2764991 Y 2822307,"[derechos fundamentales a la estabilidad, dec..."
3,T-3229258,"[ley 797 de 2003, derechos fundamentales a la..."
4,T-2828671,"[derechos fundamentales al trabajo, decreto 2..."
...,...,...
784,T-7349929,"[derechos fundamentales a la salud, decreto 2..."
785,T-7277620,"[ley 906 de 2004, derecho a la vivienda, decre..."
786,T-7139120,"[ley 100 de 1993, decreto 2591 de 1991, derech..."
787,T-7163048,"[ley 963 de 2005, derecho fundamental a la tut..."


### tags decision

In [121]:
all_tags = [t.lower() for tags in decision_df['text'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

tags 991 / 2367


### tags antecedentes

In [122]:
all_tags = [t.lower() for tags in antecedentes_df['text'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

tags 665 / 1197


## reglas

In [123]:
all_tags = [[t.lower() for t in tags] for tags in decision_df['text'][1:3]]
all_tags

[['decreto 2591 de 1991decreto 2591 de 1991',
  'ley 1098 de 2006',
  'derecho a la igualdad'],
 ['derechos fundamentales a la  estabilidad',
  'decreto 2591 de 1991decreto 2591 de 1991, decreto 3135 de 1968decreto 3135 de 1968, decreto 4588 de 2006decreto 4588 de 2006, decreto 468 de 1990decreto 468 de 1990, decreto 4588 de 2006decreto 4588 de 2006',
  'ley 74 de 1968']]

In [124]:
all_tags = [[t.lower() for t in tags] for tags in parquet_data['decretos'][10:11]]
all_tags

[['decreto 992 de 1930, normas que no estaban vigentes  ya que habian sido subrogadas por el decreto ley 1355 de 1970',
  'decreto 992 de 1930 fueron subrogados por  el decreto ley 1355 de 1970',
  'decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970',
  'decreto ley 1355 de 1970 en su articulo 125 y siguientes,  regulo integralmente la materia a que se referia aquel',
  'decreto ley 1355 de 1970 y  subsidiariamente por los codigos departamentales y distritales de policia']]

In [204]:
all_tags = [[t.lower() for t in tags] for tags in decision_df['text'][1:30]]
itemsets, reglas = apriori(all_tags, min_support=0.001, min_confidence=0.01)
count = 0
dict_rules = {}
dict_rules_metrics = {}
for regla in reglas:
    list_consecuente = []
    count = count + 1
    if count > 20:
        break
    if regla.confidence > 0.8:
        tem = dict_rules.get(regla.lhs[0], list())
        dict_rules.update({regla.lhs[0]: tem})
        tem = dict_rules.get(regla.lhs[0])
        tem.append(regla.rhs[0])
        dict_rules.update({regla.lhs[0]: tem})
        dict_rules_metrics.update({
            regla.lhs[0]: {
                "consequent": dict_rules[regla.lhs[0]],
                "metrics": {
                    "conf": regla.confidence,
                    "supp": regla.support,
                    "lift": regla.lift,
                    "conv": regla.conviction 
                }
            }
        })
        print(regla)
print("")
print("==========AGRUPACION==============")
print("")
pprint.pprint(dict_rules_metrics)

{derecho a la igualdad} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.069, lift: 1.208, conv: 172413793.103)
{ley 1098 de 2006} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.034, lift: 1.208, conv: 172413793.103)
{ley 1098 de 2006} -> {derecho a la igualdad} (conf: 1.000, supp: 0.034, lift: 14.500, conv: 931034482.759)
{derechos fundamentales a la  estabilidad} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.034, lift: 1.208, conv: 172413793.103)
{ley 74 de 1968} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.034, lift: 1.208, conv: 172413793.103)
{ley 74 de 1968} -> {derechos fundamentales a la  estabilidad} (conf: 1.000, supp: 0.034, lift: 29.000, conv: 965517241.379)
{derechos fundamentales a la  estabilidad} -> {ley 74 de 1968} (conf: 1.000, supp: 0.034, lift: 29.000, conv: 965517241.379)
{derechos fundamentales a la vida} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.069, lift: 1.208, conv: 172413793.103)
{ley 797  de 2003} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.03

In [203]:
all_tags = [[t.lower() for t in tags] for tags in antecedentes_df['text'][10:40]]
itemsets, reglas = apriori(all_tags, min_support=0.001, min_confidence=0.01)
count = 0
dict_rules = {}
dict_rules_metrics = {}
for regla in reglas:
    list_consecuente = []
    count = count + 1
    if count > 20:
        break
    if regla.confidence > 0.8:
        tem = dict_rules.get(regla.lhs[0], list())
        dict_rules.update({regla.lhs[0]: tem})
        tem = dict_rules.get(regla.lhs[0])
        tem.append(regla.rhs[0])
        dict_rules.update({regla.lhs[0]: tem})
        dict_rules_metrics.update({
            regla.lhs[0]: {
                "consequent": dict_rules[regla.lhs[0]],
                "metrics": {
                    "conf": regla.confidence,
                    "supp": regla.support,
                    "lift": regla.lift,
                    "conv": regla.conviction 
                }
            }
        })
        print(regla)
print("")
print("==========AGRUPACION==============")
print("")
pprint.pprint(dict_rules_metrics)

{derechos  fundamentales al minimo} -> {decreto 3770  de 2008} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{decreto 3770  de 2008} -> {derechos  fundamentales al minimo} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{ley 70 de 1993} -> {decreto 3770  de 2008} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{decreto 3770  de 2008} -> {ley 70 de 1993} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{ley 70 de 1993} -> {derechos  fundamentales al minimo} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{derechos  fundamentales al minimo} -> {ley 70 de 1993} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{derechos  fundamentales a la seguridad} -> {decreto 691 de 1994} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{decreto 691 de 1994} -> {derechos  fundamentales a la seguridad} (conf: 1.000, supp: 0.033, lift: 30.000, conv: 966666666.667)
{decreto 691 de 1994} -> {ley 100 de 1993} (co

# Reglas de asociacion / exploratorio

In [1]:
all_tags = [t.lower() for tags in parquet_data['folios'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

NameError: name 'parquet_data' is not defined

In [25]:
all_tags = [t.lower() for tags in parquet_data['articulos'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

tags 12513 / 16156


In [26]:
all_tags = [t.lower() for tags in parquet_data['sentencias'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

tags 22478 / 26896


In [27]:
all_tags = [t.lower() for tags in parquet_data['decretos'].values for t in tags]
tags, cnt = np.unique(all_tags, return_counts=True)
print("tags",len(tags), "/",len(all_tags))

tags 3888 / 5574


In [28]:
list_data = parquet_data['decretos'][10:11]
list_data = list(list_data.iloc[0])

In [29]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(list_data)]
tagged_data

[TaggedDocument(words=['decreto', '992', 'de', '1930', ',', 'normas', 'que', 'no', 'estaban', 'vigentes', 'ya', 'que', 'habian', 'sido', 'subrogadas', 'por', 'el', 'decreto', 'ley', '1355', 'de', '1970'], tags=['0']),
 TaggedDocument(words=['decreto', '992', 'de', '1930', 'fueron', 'subrogados', 'por', 'el', 'decreto', 'ley', '1355', 'de', '1970'], tags=['1']),
 TaggedDocument(words=['decreto', '992', 'de', '1930', ',', 'normatividad', 'subrogada', 'y', 'modificada', 'por', 'el', 'decreto', 'ley', '1355', 'de', '1970'], tags=['2']),
 TaggedDocument(words=['decreto', 'ley', '1355', 'de', '1970', 'en', 'su', 'articulo', '125', 'y', 'siguientes', ',', 'regulo', 'integralmente', 'la', 'materia', 'a', 'que', 'se', 'referia', 'aquel'], tags=['3']),
 TaggedDocument(words=['decreto', 'ley', '1355', 'de', '1970', 'y', 'subsidiariamente', 'por', 'los', 'codigos', 'departamentales', 'y', 'distritales', 'de', 'policia'], tags=['4'])]

In [118]:
list_data[1:3]

['decreto 992 de 1930 fueron subrogados por  el decreto ley 1355 de 1970',
 'decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970']

In [106]:
tagged_data[1][0]

['decreto',
 '992',
 'de',
 '1930',
 'fueron',
 'subrogados',
 'por',
 'el',
 'decreto',
 'ley',
 '1355',
 'de',
 '1970']

In [53]:
parquet_data['decretos'][10:13]

10    [decreto 992 de 1930, normas que no estaban vi...
11                                                   []
12                               [decreto 2591 de 1991]
Name: decretos, dtype: object

In [114]:
all_tags = [[t.lower() for t in tags] for tags in parquet_data['decretos'][10:11]]
type(all_tags)
all_tags
# itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
# for r in rules:
#     if r.confidence > 0.8:
#         print(r)

[['decreto 992 de 1930, normas que no estaban vigentes  ya que habian sido subrogadas por el decreto ley 1355 de 1970',
  'decreto 992 de 1930 fueron subrogados por  el decreto ley 1355 de 1970',
  'decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970',
  'decreto ley 1355 de 1970 en su articulo 125 y siguientes,  regulo integralmente la materia a que se referia aquel',
  'decreto ley 1355 de 1970 y  subsidiariamente por los codigos departamentales y distritales de policia']]

In [48]:
all_tags = [[t.lower() for t in tags] for tags in parquet_data['decretos'][10:11]]
type(all_tags)
all_tags
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
count = 0
for r in rules:
    count = count + 1
    if count > 10:
        break
    if r.confidence > 0.8:
        print(r)

{decreto 992 de 1930 fueron subrogados por  el decreto ley 1355 de 1970} -> {decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970} -> {decreto 992 de 1930 fueron subrogados por  el decreto ley 1355 de 1970} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{decreto 992 de 1930, normas que no estaban vigentes  ya que habian sido subrogadas por el decreto ley 1355 de 1970} -> {decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{decreto  992  de 1930, normatividad subrogada y modificada por el decreto ley 1355 de 1970} -> {decreto 992 de 1930, normas que no estaban vigentes  ya que habian sido subrogadas por el decreto ley 1355 de 1970} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{decreto ley 1355 de 197

In [127]:

all_tags = [tagged_data[1][0]]#[[t.lower() for t in tags] for tags in tagged_data[1][0]]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
count = 0
for r in rules:
    if r.confidence > 0.8:
        count = count + 1
        if count > 10:
            break;
        print(r)

{1930} -> {1355} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1355} -> {1930} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1970} -> {1355} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1355} -> {1970} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{992} -> {1355} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1355} -> {992} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{de} -> {1355} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1355} -> {de} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{decreto} -> {1355} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)
{1355} -> {decreto} (conf: 1.000, supp: 1.000, lift: 1.000, conv: 0.000)


In [117]:
dec['decretos'].values

array([list(['decreto 2591 de 1991']),
       list(['decreto 2591 de  1991, para los efectos alli contemplados', 'decreto 2591 de 1991 contempla: () la accion de tutela no  procedera: 1']),
       list(['decreto 2591 de 1991', 'decreto  2591 de 1991, siendo objeto de analisis de constitucionalidad en la sentencia  c-134 de 1994, en la que se declararon inexequibles algunos de sus apartes y  exequibles otros[8], bajo el entendido de que  la accion de tutela procede siempre contra el particular que este prestando  cualquier servicio publico, y por violacion de cualquier derecho constitucional  fundamental']),
       ...,
       list(['decreto 2591 de 1991', 'decreto 1730 de 2001: incompatibilidad', 'decreto 1295 de 1994, las indemnizaciones sustitutivas de  vejez y de invalidez, son incompatibles con las pensiones de vejez y de  invalidez', 'decreto 758 de 1990), derogado por la ley 100 de  1993, que, a su vez, en este aspecto, fue modificada por la ley 797 de 2003-o  de un regimen anter

In [145]:
parquet_data[1:5]

,folios,articulos,sentencias,decretos,reference
1,"[articulo 36 del decreto 2591 de 1991, para l...","[articulo 36 del decreto 2591 de 1991, para l...",[sentencia proferida por el juzgado primero pe...,"[decreto 2591 de 1991, para los efectos alli ...",T-3176539
2,[articulos 86 y 241-9 de la constitucion poli...,[articulos 86 y 241-9 de la constitucion poli...,"[sentencia c-134 de 1994, en la que se declar...","[decreto 2591 de 1991, decreto 2591 de 1991, ...",T-3180007
3,[articulo 239 del codigo sustantivo del trabaj...,[articulo 239 del codigo sustantivo del trabaj...,[sentencia proferida el siete (7) de mayo de 2...,"[decreto 2591 de 1991, decreto 3135 de 1968, e...",T-2764991 Y 2822307
4,[articulos 86 y 241 numeral 9° de la constituc...,[articulos 86 y 241 numeral 9° de la constituc...,[sentencia proferida dentro del proceso de la ...,[decreto 2591 de 1991],T-3201980


In [133]:
all_tags = [[t.lower() for t in tags] for tags in dec['decretos'].values]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.2)
count = 0
for r in rules:
    if r.confidence > 0.8:
        count = count + 1
        if count > 1:
            break;
        print(r)

{decreto 3135 de 1968, en el entendido de que,  en los terminos de esta sentencia, y debido al principio de igualdad (cp art} -> {decreto 2591 de 1991} (conf: 1.000, supp: 0.002, lift: 2.685, conv: 627570872.707)


In [96]:
all_tags = [[t.lower() for t in tags] for tags in dec['decretos'].values]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
for r in rules:
    if r.confidence > 0.2:
        print(r)

In [148]:
all_tags = [[t.lower() for t in tags] for tags in dec['articulos'].values]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
print(len(rules))
# for r in rules:
#     if r.confidence > 0.8:
#         print(r)

353604


In [ ]:
all_tags = [[t.lower() for t in tags] for tags in dec['sentencias'].values]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
print(len(rules))
for r in rules:
    if r.confidence > 0.8:
        print(r)

In [ ]:
all_tags = [[t.lower() for t in tags] for tags in dec['folios'].values]
itemsets, rules = apriori(all_tags, min_support=0.001, min_confidence=0.01)
print(len(rules))
for r in rules:
    if r.confidence > 0.8:
        print(r)